# ETL YELP

# 1. Importar librerias

In [2]:
import pandas as pd
import numpy as np
import re
import requests
import warnings
warnings.filterwarnings("ignore", message="Downcasting.*", category=FutureWarning)


# 2. Carga de Datos

In [4]:
dfBusinessYelp = pd.read_csv('Data/Business.csv')


C:\Users\U ANGEL\AppData\Local\Temp\ipykernel_16500\3750625278.py:1: DtypeWarning: Columns (14,15,16,17,18,19,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBusinessYelp = pd.read_csv('Data/Business.csv')


In [ ]:
Review = pd.read_csv('Data/Review-1.csv')
Tips = pd.read_csv('Data/Tips.csv')
User = pd.read_csv('Data/User.csv')

# 3. Exploracion y Limpieza de Datos

## 3.1 Dataset Business Yelp

### 3.1.1. Exploración inicial de los datos

In [5]:
#Vizualizo las primeras filas del DataFrame
dfBusinessYelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state.1,postal_code.1,latitude.1,longitude.1,stars.1,review_count.1,is_open.1,attributes.1,categories.1,hours.1
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Obtener información sobre las columnas y los tipos de datos
dfBusinessYelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   business_id     150346 non-null  object 
 1   name            150346 non-null  object 
 2   address         145219 non-null  object 
 3   city            150346 non-null  object 
 4   state           150343 non-null  object 
 5   postal_code     150273 non-null  object 
 6   latitude        150346 non-null  float64
 7   longitude       150346 non-null  float64
 8   stars           150346 non-null  float64
 9   review_count    150346 non-null  int64  
 10  is_open         150346 non-null  int64  
 11  attributes      136602 non-null  object 
 12  categories      150243 non-null  object 
 13  hours           127123 non-null  object 
 14  business_id.1   5 non-null       object 
 15  name.1          5 non-null       object 
 16  address.1       5 non-null       object 
 17  city.1    

In [ ]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfBusinessYelp.isnull().sum()
print(filas_vacias_por_columna)

business_id            0
name                   0
address             5127
city                   0
state                  3
postal_code           73
latitude               0
longitude              0
stars                  0
review_count           0
is_open                0
attributes         13744
categories           103
hours              23223
business_id.1     150341
name.1            150341
address.1         150341
city.1            150341
state.1           150341
postal_code.1     150341
latitude.1        150341
longitude.1       150341
stars.1           150341
review_count.1    150341
is_open.1         150341
attributes.1      150341
categories.1      150341
hours.1           150341
dtype: int64


Al evaluar la cantidad de nulos por columnas vemos que las columnas : 'business_id.1', 'name.1',
       'address.1', 'city.1', 'state.1', 'postal_code.1', 'latitude.1',
       'longitude.1', 'stars.1', 'review_count.1', 'is_open.1', 'attributes.1',
       'categories.1', 'hours.1', estan llenos de nulos

In [6]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfBusinessYelp.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


Como logramos observas no encontramos valores duplicados en el DataFrame

In [7]:
dfBusinessYelp['state'].unique()

array([nan, 'CA', 'MO', 'AZ', 'PA', 'TN', 'FL', 'IN', 'LA', 'AB', 'NV',
       'ID', 'DE', 'IL', 'NJ', 'NC', 'CO', 'WA', 'HI', 'UT', 'TX', 'MT',
       'MI', 'SD', 'XMS', 'MA', 'VI', 'VT'], dtype=object)

Al ver que hay valores nulos, siglas de los diferentes estados concluyo que la columna 'state' es mejor no usarla y sacar los estados mediante la longitud y latitud

### 3.1.2. Limpieza y Normalización de datos

#### Filtrado de datos

In [8]:
# El alcance del proyecto se limita a los diferentes negocios 
# dentro de la categoria 'Nail Salon', por lo que se aplica el filtro.

# Eliminar filas con valores NaN en la columna 'categories'
dfBusinessYelp = dfBusinessYelp.dropna(subset=['categories'])

# Filtrar las filas que contienen 'Nail Salons' en la columna 'categories'
dfBusinessYelp = dfBusinessYelp[dfBusinessYelp['categories'].str.contains('Nail Salons')]


De limitamos el proyecto basado en la Categoria 'Nail Salon' o 'Salon de Uñas'

In [9]:
dfBusinessYelp.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state.1,postal_code.1,latitude.1,longitude.1,stars.1,review_count.1,is_open.1,attributes.1,categories.1,hours.1
74,9Rww8yE6Dm4dSMEq09nwXg,Holly Nails & Spa,9101 Belcher Rd,Pinellas Park,TN,33782,27.855347,-82.736308,4.5,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,my0bmPD5dgDFE1ia__LNlw,Nails & Beauty Lounge,953 E Brandon Blvd,Brandon,IN,33511,27.937166,-82.269959,3.5,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,CLEWowfkj-wKYJlQDqT1aw,Blow Bar Express Styling Salon,"310 S Dale Mabry Hwy, Ste 110",Tampa,TN,33609,27.941975,-82.505307,4.0,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Limpieza y Normalizacion

In [ ]:
columnas_a_eliminar = ['is_open', 'categories','state', 
       'business_id.1', 'name.1', 'address.1', 'city.1', 'state.1', 
       'postal_code.1', 'latitude.1', 'longitude.1', 'stars.1', 'review_count.1',
        'is_open.1', 'attributes.1', 'categories.1', 'hours.1']

# Eliminar las columnas especificadas
dfBusinessYelp = dfBusinessYelp.drop(columnas_a_eliminar, axis=1)

Como mencionamos en el conteo de nulos eliminamos columnas con demasiados valores vacios que no nos ayudaran para realizar un buen analisis.
Las columnas'is_open','address', 'name' ya tenemos los valores y se filtro de manera correcta asi que seria redundante tener esas columnnas.
Con respecto a la columna 'state' tiene valores que no corresponde con la latitud y longitud en el mapa, asi que mejor es no contar con esa columna.

In [ ]:
# Define una función para obtener el estado desde la latitud y longitud
def reverse_geocode(lat, lon):
    # Aquí va tu código para obtener el estado desde la latitud y longitud
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}"
    response = requests.get(url)
    data = response.json()
    if 'address' in data:
        state = data['address'].get('state', '')
        return state

# Aplica la función a cada fila del DataFrame y crea una lista de resultados
states = [reverse_geocode(row['latitude'], row['longitude']) for _, row in dfBusinessYelp.iterrows()]

# Asigna la lista de estados a la columna 'state' del DataFrame
dfBusinessYelp['state'] = states

Usando la API OpenStreetMap (OSM) se realizo una geocodificacion inversa que me devuelve los diferentes estados('state') basado en las columnas 'latitude' y 'longitude' 

In [ ]:
# Aplicamos el filtro con referencia en la columna 'name'
dfBusinessYelp = dfBusinessYelp[dfBusinessYelp['state'] == 'Florida']

Delimitamos los datos a un solo estado 'Florida'

In [ ]:
# Crea un nuevo DataFrame solo con las columnas 'business_id' y 'opening_hours_id'
Atributes = dfBusinessYelp[['business_id','attributes']]

# Crea un nuevo DataFrame solo con las columnas 'business_id' y 'opening_hours_id'
opening_hours = dfBusinessYelp[['business_id', 'hours']]

Hacemos una copia para generar las tablas Attributes y Opening Hours

In [ ]:
# Utilizo el método drop() de pandas para eliminar las columnas especificadas
dfBusinessYelp = dfBusinessYelp.drop(['attributes', 'hours', 'state'], axis=1)

Borramos el estado 'Florida' debido que todos los salones de uña 'Nail Salon' pertenecen al mismo estado

In [ ]:
dfBusinessYelp.head()

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,hours,source
74,9Rww8yE6Dm4dSMEq09nwXg,Holly Nails & Spa,9101 Belcher Rd,Pinellas Park,33782,27.855347,-82.736308,4.5,16,"{'Tuesday': '9:30-19:0', 'Wednesday': '9:30-19...",Y
250,my0bmPD5dgDFE1ia__LNlw,Nails & Beauty Lounge,953 E Brandon Blvd,Brandon,33511,27.937166,-82.269959,3.5,84,"{'Monday': '9:30-19:0', 'Wednesday': '9:30-19:...",Y
294,CLEWowfkj-wKYJlQDqT1aw,Blow Bar Express Styling Salon,"310 S Dale Mabry Hwy, Ste 110",Tampa,33609,27.941975,-82.505307,4.0,55,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '...",Y
362,gbetDRjllP5cOxaWxFjGHA,Pretty Nails,5316 Little Rd,New Port Richey,34655,28.237112,-82.668024,3.0,6,NaN,Y
630,FYzZd2mlMIPScy95UY2xkw,McMahan Hair Salon,10464 Roosevelt Blvd N,St. Petersburg,33716,27.866938,-82.645031,4.0,11,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', '...",Y


In [ ]:
Business = dfBusinessYelp

## 3.2 Dataset Reviews Yelp

### 3.2.1. Exploración inicial de los datos

In [ ]:
#Vizualizo las primeras filas del DataFrame
Review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07
2,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5,0,0,0,My boyfriend and I tried this deli for the fir...,2018-08-23 21:39:38
3,E9AB7V4z8xrt2uPF7T55FQ,iYY5Ii1LGpZCpXFkHlMefw,Zx7n8mdt8OzLRXVzolXNhQ,5,0,0,0,Amazing biscuits and (fill in the blank). Grea...,2018-04-27 23:03:21
4,A4n4YaE-owOVgTQcrVqHUw,S7bjj-L07JuRr-tpX1UZLw,I6L0Zxi5Ww0zEWSAVgngeQ,4,0,0,0,The cafe was extremely cute. We came at 8am an...,2018-07-07 20:50:12


In [ ]:
Review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678292 entries, 0 to 2678291
Data columns (total 9 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   user_id      object
 2   business_id  object
 3   stars        int64 
 4   useful       int64 
 5   funny        int64 
 6   cool         int64 
 7   text         object
 8   date         object
dtypes: int64(4), object(5)
memory usage: 183.9+ MB


In [ ]:
# Contar filas vacías por columna
filas_vacias_por_columna = Review.isnull().sum()
print(filas_vacias_por_columna)

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64


Como observamos no tenemos valores nulos

In [ ]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = Review.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


Como observamos no tenemos filas duplicadas

### 3.2.2. Limpieza y Normalización de datos

Filtrado de datos

In [ ]:
Business = pd.read_csv('Data/Cleaned_Business_Y.csv')

In [ ]:
# Obtener los id únicos del DataFrame business
business_ids_to_keep = Business['business_id'].unique()


# Friltamos el dfreviewYelp basado en los balores unico de lao id de dfbusinessYelp
Review = Review[Review['business_id'].isin(business_ids_to_keep)]

# El conjunto de datos se filtró por la fecha desde el inicio del 2016 hasta finalizar el 2021.
#dfreviewYelp = dfreviewYelp[(dfreviewYelp['date'] >= '2016-01-01') & (dfreviewYelp['date'] >= '2021-12-31')]

Aplicamos un filtro solo para quedarnos con los valores que coincidan con el id del negocio

#### Limpieza y Normalizacion 

In [ ]:
# Convertir la columna 'date' a tipo datetime
Review['date'] = pd.to_datetime(Review['date'])

In [ ]:
columnas_a_eliminar = ['useful', 'funny', 'cool']

# Eliminar las columnas especificadas
Review = Review.drop(columnas_a_eliminar, axis=1)

Elinamos las siguientes columnas 'useful', 'funny', 'cool' debido que no tiene una guia o rango que me diga si esta bien o no, 'date' lo dividimos en años , mes, dia asi que no lo necesitamos

In [ ]:
# Aplicar la limpieza de texto directamente sobre la columna 'text'
Review['text'] = Review['text'].str.lower().str.replace(r'[^a-z0-9\s]', '')

Convertimos todo los valores de la columna 'text' a minuscula para guiarnos mejor

In [ ]:
# Modificamos el tipo de dato a un valor flotante
Review['stars'] = Review['stars'].astype('float64')

Transforme la columna 'stars' para tener similitud del tipo de dato con otra tabla

In [ ]:
Review = Review.rename(columns={'stars' : 'review_stars'})

Renombrom stars para indentificar a que se refiere esa calificacion

In [ ]:
Review.head(3)

,review_id,user_id,business_id,review_stars,text,date
51,Blzra0ClBO8mORAnUCk5Xg,6Mhbp06FEhD9uxUGCCUmKA,fBNjTSdPJ1mtTzIHYpQnDw,1.0,zero stars!! this place is dirty and disorgani...,2018-09-03 13:04:03
96,cbYsdDN5bomxCXCNnMps6g,-FFgJuDZLmcZtrxDTRC7sg,my0bmPD5dgDFE1ia__LNlw,5.0,this is the place we have been searching for! ...,2018-08-23 22:28:46
556,qaisR2xTb-ybLv9GVB-whg,g34LsuGf9gj9h5wqhye-Jw,bP8Y5cHMda64Z3tGA-cYzw,5.0,i found this place through yelp. i was attract...,2018-03-23 19:29:16


## 3.3 Dataset User 

### 3.3.1. Exploración inicial de los datos

In [ ]:
User.head(3)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18


In [ ]:
User.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


In [ ]:
# Contar filas vacías por columna
filas_vacias_por_columna = User.isnull().sum()
print(filas_vacias_por_columna)

user_id                     0
name                       41
review_count                0
yelping_since               0
useful                      0
funny                       0
cool                        0
elite                 1988836
friends                878551
fans                        0
average_stars               0
compliment_hot              0
compliment_more             0
compliment_profile          0
compliment_cute             0
compliment_list             0
compliment_note             0
compliment_plain            0
compliment_cool             0
compliment_funny            0
compliment_writer           0
compliment_photos           0
dtype: int64


Como logramos Oberservar las columnas 'elite' y 'friends' tienen muchos valores vacios

In [ ]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = User.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 117700


La tabla no cuenta con valores duplicados

### 3.3.2. Limpieza y Normalización de datos

In [ ]:
# Obtener los id únicos del DataFrame dfbusinessYelp
business_ids_to_keep = Review['user_id'].unique()


# Friltamos el dfreviewYelp basado en los balores unico de lao id de dfbusinessYelp
User = User[User['user_id'].isin(business_ids_to_keep)]

# El conjunto de datos se filtró por la fecha desde el inicio del 2016 hasta finalizar el 2021.
#dfreviewYelp = dfreviewYelp[(dfreviewYelp['date'] >= '2016-01-01') & (dfreviewYelp['date'] >= '2021-12-31')]

In [ ]:
# Eliminar filas duplicadas del DataFrame User
User = User.drop_duplicates()

Aplicamos un filtro solo para quedarnos con los valores que coincidan con el id del negocio

In [ ]:
#Creo la tabla Compliment
Compliment = User[['user_id','useful', 'funny', 'cool', 'compliment_profile']].copy()

Creo la nueva tabla Compliment

In [ ]:
#Columnas a eliminar de la tabla User
columnas_a_eliminar = ['elite', 'friends', 'compliment_hot','compliment_more', 'compliment_profile',
         'compliment_cute', 'compliment_list', 'compliment_note', 
         'compliment_plain', 'compliment_cool', 'compliment_funny', 
         'compliment_writer', 'compliment_photos','useful', 'funny', 'cool']

# Eliminar las columnas especificadas
User = User.drop(columnas_a_eliminar, axis=1)

Borramos columnas que tienen datos nulos y otras columnas que copiamos para normalizar y dar mejor performance a la tabla

In [ ]:
User.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,fans,average_stars
975,j4cmflYbwAktOxZ_iiJCOQ,Danielle,236,2009-08-09 13:37:21,1276,717,1040,44,3.81
1242,pou3BbKsIozfH50rxmnMew,Brett,2812,2012-01-11 17:19:55,22586,8950,14927,914,4.28
1857,3ywXGq4PyUoR_XXDvK_RIA,Toni,729,2006-06-13 03:44:01,1875,689,1268,77,4.20
1936,rTJ3EecGoQNbfFj7mqjDmw,Elizabeth,529,2009-03-18 03:49:43,796,189,296,37,3.74
2079,G6uiBIcMc21mxtIsR6K_3w,Irene,486,2009-12-14 01:40:43,2296,1016,1106,106,3.38


In [ ]:
# Utilizar la función melt para convertir las columnas en filas
Compliment = pd.melt(Compliment, id_vars=['user_id'], var_name='name_compliment', value_name='counts')

# Ordenar el DataFrame por user_id
Compliment = Compliment.sort_values(by='user_id')

In [ ]:
# Creamos la tabla name_compliment
data = {
    'compliment_id': ['com-001', 'com-002', 'com-003', 'com-004'],
    'name_compliment': ['useful', 'funny', 'cool', 'compliment_profile']}

name_compliment = pd.DataFrame(data)

In [ ]:
# Unir las tablas
Compliment = pd.merge(Compliment, name_compliment, left_on='name_compliment', right_on='name_compliment')

# Eliminar la columna 'compliment_name'
Compliment = Compliment.drop(columns=['name_compliment'])

In [ ]:
# Ordeno las columnas
Compliment = Compliment[['user_id', 'compliment_id', 'counts']]

In [ ]:
Compliment.head()

## 3.4 Dataset Attributes

### 3.4.1. Exploración inicial de los datos

In [ ]:
Atributes.head()

,business_id,attributes
0,9Rww8yE6Dm4dSMEq09nwXg,"{'BusinessAcceptsCreditCards': 'True', 'BikePa..."
1,my0bmPD5dgDFE1ia__LNlw,"{'ByAppointmentOnly': 'False', 'BusinessAccept..."
2,CLEWowfkj-wKYJlQDqT1aw,"{'BusinessParking': ""{'garage': False, 'street..."
3,gbetDRjllP5cOxaWxFjGHA,NaN
4,FYzZd2mlMIPScy95UY2xkw,"{'BikeParking': 'True', 'BusinessAcceptsCredit..."


In [ ]:
Atributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877 entries, 0 to 876
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  877 non-null    object
 1   attributes   839 non-null    object
dtypes: object(2)
memory usage: 13.8+ KB


In [ ]:
# Contar filas vacías por columna
filas_vacias_por_columna = Atributes.isnull().sum()
print(filas_vacias_por_columna)

business_id     0
attributes     38
dtype: int64


In [ ]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = Atributes.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


### 3.4.2. Limpieza y Normalización de datos

#### Normalizacion de datos

In [ ]:
# Rellenar los valores nulos con un valor predeterminado (por ejemplo, una cadena vacía)
Atributes['attributes'] = Atributes['attributes'].fillna('False')

In [ ]:
# Convertir la columna 'attributes' en un DataFrame separado
Attributes = Atributes['attributes'].apply(eval).apply(pd.Series)

# Agregar la columna 'business_id' de nuevo al DataFrame de atributos
Attributes['business_id'] = Atributes['business_id']

# Reorganizar las columnas para colocar 'business_id' al principio
Attributes = Attributes[['business_id'] + list(Attributes.columns[:-1])]

La columna attributes tienes valores en forma de diccionarios y lo desanido aplicando la funcion apply(eval) 

In [ ]:
columnas_a_eliminar = ['AcceptsInsurance','RestaurantsDelivery', 'GoodForMeal', 'RestaurantsAttire', 
                       'RestaurantsTableService', 'DogsAllowed', 'RestaurantsGoodForGroups', 'Music', 
                       'OutdoorSeating', 'HasTV', 'RestaurantsReservations', 'Alcohol', 'Ambience', 'HappyHour',
                       'GoodForDancing', 'Smoking', 'NoiseLevel', 'AcceptsInsurance', 'HairSpecializesIn', 
                       'RestaurantsTakeOut', 0, 'BusinessAcceptsBitcoin','GoodForKids','WheelchairAccessible']
# Eliminar las columnas especificadas
Attributes = Attributes.drop(columnas_a_eliminar, axis=1)

In [ ]:
from ast import literal_eval
# Remplazar los valores nan con un diccionario vacío
Attributes['BusinessParking'].fillna('{}', inplace=True)

# Convertir la columna 'BusinessParking' a un diccionario
Attributes['BusinessParking'] = Attributes['BusinessParking'].apply(literal_eval)

# Desanidar el diccionario
CarParking = pd.json_normalize(Attributes['BusinessParking'])

In [ ]:
# Verificar si al menos una de las columnas tiene True en cada fila
CarParking['Parking'] = CarParking[['garage', 'street', 'validated', 'lot', 'valet']].any(axis=1)

In [ ]:
# Eliminar las columnas no deseadas de la tabla parking
CarParking = CarParking.drop(['garage', 'street', 'validated', 'lot', 'valet'], axis=1)

In [ ]:
# Combinar la tabla parking con la tabla Attributes
Attributes = Attributes.merge(CarParking, left_index=True, right_index=True)

In [ ]:
# Eliminar la columna 'BusinessParking' de la tabla Attributes
Attributes = Attributes.drop('BusinessParking', axis=1)

In [ ]:
def normalize_wifi(value):
    if value in ["u'free'", "'free'"]:
        return True
    else:
        return False

# Aplicar la función a la columna 'WiFi'
Attributes['WiFi'] = Attributes['WiFi'].apply(normalize_wifi)

In [ ]:
# Crear un diccionario de mapeo
mapping = {2.0: True, 1.0: True, 3.0: True, 4.0: True}

# Mapear los valores y convertir a booleanos
Attributes['RestaurantsPriceRange2'] = Attributes['RestaurantsPriceRange2'].map(mapping).astype(bool)

In [ ]:
# Reemplazar NaN por False en el DataFrame Attributes
Attributes = Attributes.fillna(False)

In [ ]:
# Renombrar la columna 'RestaurantsPriceRange2' a 'Food'
Attributes = Attributes.rename(columns={'RestaurantsPriceRange2': 'Food'})

In [ ]:
Attributes = Attributes.rename(columns={'Parking': 'CarParking'})

In [ ]:
# Agregar una columna 'Attribute_id' que contiene el ID único para cada fila
Attributes['Attribute_id'] = ['at-' + str(i).zfill(3) for i in range(1, len(Attributes) + 1)]

In [ ]:
# Lista con el orden deseado de las columnas
column_order = ['business_id', 'Attribute_id', 'BusinessAcceptsCreditCards', 'CarParking', 'BikeParking', 'WiFi', 'Food', 'ByAppointmentOnly']

# Reordenar las columnas del DataFrame
Attributes = Attributes.reindex(columns=column_order)

In [ ]:
# Convertir las columnas de tipo booleano a tipo objeto (str)
Attributes['carparking'] = Attributes['carparking'].astype(str)
Attributes['wifi'] = Attributes['wifi'].astype(str)
Attributes['food'] = Attributes['food'].astype(str)

In [ ]:
# Lista de atributos
attributes = ['businessacceptscreditcards', 'carparking', 'bikeparking', 'wifi', 'food', 'byappointmentonly']

# Crear un nuevo DataFrame
df_new = pd.DataFrame(columns=['business_id', 'attributes', 'value'])

# Iterar sobre cada fila del DataFrame original
for index, row in Attributes.iterrows():
    business_id = row['business_id']
    for attribute in attributes:
        value = row[attribute]
        df_new = pd.concat([df_new, pd.DataFrame({'business_id': [business_id], 'attributes': [attribute], 'value': [value]})], ignore_index=True)

Attributes = df_new

In [ ]:
# Filtrar el DataFrame para mantener solo las filas donde el valor es True
Attributes = Attributes[Attributes['value'] == 'True']

In [ ]:
Attributes = Attributes.drop(['value'], axis = 1)

In [ ]:
Attributes.head()

,business_id,attributes
0,9Rww8yE6Dm4dSMEq09nwXg,businessacceptscreditcards
1,9Rww8yE6Dm4dSMEq09nwXg,carparking
2,9Rww8yE6Dm4dSMEq09nwXg,bikeparking
3,9Rww8yE6Dm4dSMEq09nwXg,wifi
4,9Rww8yE6Dm4dSMEq09nwXg,food


In [ ]:
# Crear un DataFrame con los nombres de los atributos
data = {
    'type_attributes_id': ['t-001', 't-002', 't-003', 't-004', 't-005', 't-006'],
    'name_attributes': ['businessacceptscreditcards', 'carparking', 'bikeparking', 'wifi', 'food', 'byappointmentonly']
}

# Crear el DataFrame
Type_Attribute = pd.DataFrame(data)

In [ ]:
# Realizar la unión por la columna 'attributes' y 'name_attributes'
df = Attributes.merge(Type_Attribute, left_on='attributes', right_on='name_attributes', how='left')

# Mantener solo las columnas necesarias
Attributes = df[['business_id', 'type_attributes_id']]

In [ ]:
# Crear una columna 'attribute_id' que sea incremental
Attributes['attributes_id'] = ['at-' + str(i).zfill(3) for i in range(1, len(Attributes) + 1)]

In [ ]:
# Ordenar las columnas
Attributes = Attributes[['attributes_id', 'business_id','type_attributes_id']]

In [ ]:
Attributes.head()

,business_id,type_attributes_id
0,9Rww8yE6Dm4dSMEq09nwXg,t-001
1,9Rww8yE6Dm4dSMEq09nwXg,t-002
2,9Rww8yE6Dm4dSMEq09nwXg,t-003
3,9Rww8yE6Dm4dSMEq09nwXg,t-004
4,9Rww8yE6Dm4dSMEq09nwXg,t-005


## 3.5 Dataset Opening_Hours

### 3.5.1. Transformacion de Datos

In [ ]:
from ast import literal_eval

# Crear una lista vacía para almacenar las filas de la nueva tabla
opening_hours = []

# Iterar sobre cada fila en la tabla 'Business'
for idx, row in Business.iterrows():
    # Extraer los horarios de operación
    hours_str = row['hours']
    try:
        # Convertir la cadena de texto a un diccionario
        hours = literal_eval(hours_str)
        # Iterar sobre cada día de la semana en los horarios de operación
        for day, time in hours.items():
            # Dividir el horario en 'hora de apertura' y 'hora de cierre'
            opening_time, closing_time = time.split('-')
            # Agregar una nueva fila a la lista
            opening_hours.append([row['business_id'], day, opening_time, closing_time])
    except (ValueError, SyntaxError):
        # Manejar el caso en el que la cadena no sea un diccionario válido
        pass

# Convertir la lista en una tabla de pandas
opening_hours = pd.DataFrame(opening_hours, columns=['business_id', 'day_of_week', 'opening_time', 'closing_time'])

In [ ]:
Days = {'days_id': ['d-001', 'd-002', 'd-003', 'd-004', 'd-005', 'd-006', 'd-007'],
    'name_days': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
}
Days = pd.DataFrame(Days)

In [ ]:
# Crear un nuevo DataFrame combinando opening_hours y Days usando merge
opening_hours = pd.merge(opening_hours, Days, left_on='day_of_week', right_on='name_days')

# Eliminar la columna 'name_days' ya que ya no es necesaria
opening_hours.drop(['name_days', 'day_of_week'], axis=1, inplace=True)


In [ ]:
# Unir las columnas opening_time y closing_time con un guión "-"
opening_hours['combined_time'] = opening_hours['opening_time'] + '-' + opening_hours['closing_time']

In [ ]:
# Copio la tabla opening_hours
hours = opening_hours.copy()

#Borro todas las columnas exceptop combined_time
hours = hours.drop(['business_id', 'opening_time', 'closing_time', 'days_id'], axis= 1)

In [ ]:
# Obtener valores únicos de la columna 'combined_time'
unique_times = hours['combined_time'].unique()

# Crear un DataFrame con los valores únicos
unique_df = pd.DataFrame({'combined_time': unique_times})

# Agregar una columna 'hours_id' con valores incrementales
unique_df['hours_id'] = ['h-' + str(i).zfill(3) for i in range(1, len(unique_df) + 1)]

hours = unique_df

In [ ]:
# Unir las tablas opening_hours y hours usando la columna 'combined_time'
result = pd.merge(opening_hours, hours[['combined_time', 'hours_id']], on='combined_time', how='left')

# Eliminar la columna 'combined_time' si no la necesitas
result.drop('combined_time', axis=1, inplace=True)

opening_hours = result

In [ ]:
opening_hours = opening_hours.drop(['opening_time', 'closing_time'], axis= 1)

In [ ]:
# Crear una columna 'opening_hours_id' que sea incremental
opening_hours['opening_hours_id'] = ['oh-' + str(i).zfill(3) for i in range(1, len(opening_hours) + 1)]

In [ ]:
# Seleccionar las columnas en el orden deseado
opening_hours = opening_hours[['opening_hours_id', 'business_id', 'days_id', 'hours_id']]

In [ ]:
opening_hours.head()

,business_id,days_id,hours_id
0,9Rww8yE6Dm4dSMEq09nwXg,d-002,h-001
1,9Rww8yE6Dm4dSMEq09nwXg,d-003,h-001
2,9Rww8yE6Dm4dSMEq09nwXg,d-004,h-001
3,9Rww8yE6Dm4dSMEq09nwXg,d-005,h-001
4,9Rww8yE6Dm4dSMEq09nwXg,d-006,h-001


In [ ]:
# Cambiar el nombre de la columna 'combined_time' a 'Horario'
hours = hours.rename(columns={'combined_time': 'Schedule'})

In [ ]:
hours.head()

,Schedule,hours_id
0,9:30-19:0,h-001
1,10:0-17:0,h-002
2,11:0-17:0,h-003
3,0:0-0:0,h-004
4,8:0-16:0,h-005


# 4. Carga de Datos

In [ ]:
Business.to_csv('DataSets_Limpia/C_Business.csv', index=False)
Review.to_csv('DataSets_Limpia/C_Review.csv', index= False)
User.to_csv('DataSets_Limpia/C_User.csv', index= False)
Compliment.to_csv('DataSets_Limpia/C_Compliment_User.csv', index= False)
name_compliment.to_csv('DataSets_Limpia/C_Compliment.csv', index= False)
Attributes.to_csv('DataSets_Limpia/C_Attributes.csv', index= False)
opening_hours.to_csv('DataSets_Limpia/C_Opening_Hours.csv', index= False)
Type_Attribute.to_csv('DataSets_Limpia/C_Type_Attribute.csv', index= False)
Days.to_csv('DataSets_Limpia/C_Days.csv', index= False)
hours.to_csv('DataSets_Limpia/C_Schedule.csv', index= False)